In [1]:
import requests
from bs4 import BeautifulSoup
import re
import sklearn
import time
import numpy as np
import json
import pandas as pd
import random
import os
from newspaper import Article
# pdfminer.six
# import pdfminer
# import pdfminer.high_level
pd.set_option("max_colwidth", 400)
from pandarallel import pandarallel

In [2]:
def extract_urls(url):
    time.sleep(1)
    coverpage = requests.get(url).content
    soup = BeautifulSoup(coverpage, 'html.parser')
    rows = soup.find_all('div', class_= 'item-info-wrap')
    rows = [str(i) for i in rows]
    return rows

def extract_row(row):
    soup = BeautifulSoup(row, 'html.parser')
    title = soup.h2.text
    link = soup.h2.a['href']
    link = 'https://www.npr.org' + link if link.startswith('/') else link
    abstract = soup.p.text.split(' • ')[1]
    return pd.Series([title, abstract, link])

def extract_text(url):
    time.sleep(1)
    try:
        article = Article(url, keep_article_html=False)
        article.download()
        article.parse()
        return article.text
    except:
        return 'error'

In [3]:
prefix = 'https://www.npr.org/sections/'
suffix = '&date=3-31-2021'

world_list = ['world/' + 'archive?start=%s'%i for i in range(16,76072+1, 24)]
national_list = ['national/' + 'archive?start=%s'%i for i in range(16,155776+1, 24)]
politic_list = ['politics/' + 'archive?start=%s'%i  for i in range(16,72216+1, 24)]

# df = pd.DataFrame(world_list+ national_list+ politic_list,  columns =['url'] )
df = pd.DataFrame(politic_list,  columns =['url'] )
df.url = prefix + df.url + suffix
df

,url
0,https://www.npr.org/sections/politics/archive?start=16&date=3-31-2021
1,https://www.npr.org/sections/politics/archive?start=40&date=3-31-2021
2,https://www.npr.org/sections/politics/archive?start=64&date=3-31-2021
3,https://www.npr.org/sections/politics/archive?start=88&date=3-31-2021
4,https://www.npr.org/sections/politics/archive?start=112&date=3-31-2021
...,...
3004,https://www.npr.org/sections/politics/archive?start=72112&date=3-31-2021
3005,https://www.npr.org/sections/politics/archive?start=72136&date=3-31-2021
3006,https://www.npr.org/sections/politics/archive?start=72160&date=3-31-2021
3007,https://www.npr.org/sections/politics/archive?start=72184&date=3-31-2021


In [4]:
pandarallel.initialize(nb_workers=128,progress_bar=True)
df[['rows']] = df['url'].parallel_apply(extract_urls)
df

INFO: Pandarallel will run on 128 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


Process ForkPoolWorker-87:
Process ForkPoolWorker-31:
Process ForkPoolWorker-91:
Process ForkPoolWorker-19:
Process ForkPoolWorker-23:
Traceback (most recent call last):
  File "/home/yibo/miniconda3/envs/py36/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/yibo/miniconda3/envs/py36/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/yibo/miniconda3/envs/py36/lib/python3.6/multiprocessing/pool.py", line 119, in worker
    result = (True, func(*args, **kwds))
  File "/home/yibo/miniconda3/envs/py36/lib/python3.6/multiprocessing/pool.py", line 44, in mapstar
    return list(map(*args))
  File "/home/yibo/miniconda3/envs/py36/lib/python3.6/site-packages/pandarallel/pandarallel.py", line 64, in global_worker
    return _func(x)
  File "/home/yibo/miniconda3/envs/py36/lib/python3.6/site-packages/pandarallel/pandarallel.py", line 116, in wrapper
    **kwargs
  File "/home/yibo/m

KeyboardInterrupt: 

  File "/home/yibo/miniconda3/envs/py36/lib/python3.6/site-packages/pandarallel/pandarallel.py", line 64, in global_worker
    return _func(x)
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/yibo/miniconda3/envs/py36/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/home/yibo/miniconda3/envs/py36/lib/python3.6/multiprocessing/pool.py", line 119, in worker
    result = (True, func(*args, **kwds))
  File "/home/yibo/miniconda3/envs/py36/lib/python3.6/site-packages/pandarallel/data_types/series.py", line 20, in worker
    return series.apply(func, *args, **kwargs)
KeyboardInterrupt
  File "/home/yibo/miniconda3/envs/py36/lib/python3.6/site-packages/pandas/core/series.py", line 4213, in apply
    mapped = lib.map_infer(values, f, convert=convert_dtype)
  File "/home/yibo/miniconda3/envs/py36/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File 

In [5]:
df = df.explode('rows', ignore_index=True)
df

,url,rows
0,https://www.npr.org/sections/politics/archive?start=16&date=3-31-2021,"<div class=""item-info-wrap"">\n<div class=""item-info"">\n<h2 class=""title""><a data-metrics='{""action"":""Click Story Title"",""category"":""Story List""}' href=""https://www.npr.org/2021/03/17/978363447/texas-man-arrested-with-guns-ammo-outside-of-vice-presidential-residence"">Texas Man Arrested With Guns, Ammo Outside Of Vice Presidential Residence</a></h2>\n<p class=""teaser""><a href=""https://www.npr.or..."
1,https://www.npr.org/sections/politics/archive?start=16&date=3-31-2021,"<div class=""item-info-wrap"">\n<div class=""item-info"">\n<h2 class=""title""><a data-metrics='{""action"":""Click Story Title"",""category"":""Story List""}' href=""https://www.npr.org/2021/03/17/977842441/house-renews-violence-against-women-act-but-senate-hurdles-remain"">House Renews Violence Against Women Act, But Senate Hurdles Remain</a></h2>\n<p class=""teaser""><a href=""https://www.npr.org/2021/03/17/9..."
2,https://www.npr.org/sections/politics/archive?start=16&date=3-31-2021,"<div class=""item-info-wrap"">\n<div class=""item-info"">\n<div class=""slug-wrap"">\n<h3 class=""slug""><a href=""https://www.npr.org/sections/medical/"">Medical Treatments</a></h3>\n</div>\n<h2 class=""title""><a data-metrics='{""action"":""Click Story Title"",""category"":""Story List""}' href=""https://www.npr.org/2021/03/17/978288331/how-to-get-trump-voters-vaccinated-when-half-say-they-dont-plan-to"">How To G..."
3,https://www.npr.org/sections/politics/archive?start=16&date=3-31-2021,"<div class=""item-info-wrap"">\n<div class=""item-info"">\n<div class=""slug-wrap"">\n<h3 class=""slug""><a href=""https://www.npr.org/sections/coronavirus-live-updates/"">Coronavirus Updates</a></h3>\n</div>\n<h2 class=""title""><a data-metrics='{""action"":""Click Story Title"",""category"":""Story List""}' href=""https://www.npr.org/sections/coronavirus-live-updates/2021/03/17/978262865/white-house-announces-10..."
4,https://www.npr.org/sections/politics/archive?start=16&date=3-31-2021,"<div class=""item-info-wrap"">\n<div class=""item-info"">\n<h2 class=""title""><a data-metrics='{""action"":""Click Story Title"",""category"":""Story List""}' href=""https://www.npr.org/2021/03/17/978249859/irs-expected-to-push-back-tax-filing-deadline"">IRS Postpones Tax Filing Deadline To May 17</a></h2>\n<p class=""teaser""><a href=""https://www.npr.org/2021/03/17/978249859/irs-expected-to-push-back-tax-fili..."
...,...,...
44455,https://www.npr.org/sections/politics/archive?start=72208&date=3-31-2021,"<div class=""item-info-wrap"">\n<div class=""item-info"">\n<h2 class=""title""><a data-metrics='{""action"":""Click Story Title"",""category"":""Story List""}' href=""https://www.npr.org/templates/story/story.php?storyId=3413003"">Bush Faulted for NAACP Snub</a></h2>\n<p class=""teaser""><a href=""https://www.npr.org/templates/story/story.php?storyId=3413003""><time datetime=""2004-07-15""><span class=""date"">July 1..."
44456,https://www.npr.org/sections/politics/archive?start=72208&date=3-31-2021,"<div class=""item-info-wrap"">\n<div class=""item-info"">\n<h2 class=""title""><a data-metrics='{""action"":""Click Story Title"",""category"":""Story List""}' href=""https://www.npr.org/templates/story/story.php?storyId=3413001"">Bush Vies for More Support in Wisconsin</a></h2>\n<p class=""teaser""><a href=""https://www.npr.org/templates/story/story.php?storyId=3413001""><time datetime=""2004-07-15""><span class=""..."
44457,https://www.npr.org/sections/politics/archive?start=72208&date=3-31-2021,"<div class=""item-info-wrap"">\n<div class=""item-info"">\n<h2 class=""title""><a data-metrics='{""action"":""Click Story Title"",""category"":""Story List""}' href=""https://www.npr.org/templates/story/story.php?storyId=3399027"">Senate: No Gay Marriage Ban</a></h2>\n<p class=""teaser""><a href=""https://www.npr.org/templates/story/story.php?storyId=3399027""><time datetime=""2004-07-14""><span class=""date"">July 

In [6]:
df.to_csv('npr_politic_raw.csv', index=False)

In [4]:
df_others = pd.read_csv('npr_raw.csv', header=[0])
df_others

,url,rows
0,https://www.npr.org/sections/world/archive?start=16&date=3-31-2021,"<div class=""item-info-wrap"">\n<div class=""item-info"">\n<div class=""slug-wrap"">\n<h3 class=""slug""><a href=""https://www.npr.org/sections/middle-east/"">Middle East</a></h3>\n</div>\n<h2 class=""title""><a data-metrics='{""action"":""Click Story Title"",""category"":""Story List""}' href=""https://www.npr.org/2021/03/13/976875790/marking-10-years-of-war-in-syria"">Marking 10 Years Of War In Syria</a></h2>\n<p..."
1,https://www.npr.org/sections/world/archive?start=16&date=3-31-2021,"<div class=""item-info-wrap"">\n<div class=""item-info"">\n<div class=""slug-wrap"">\n<h3 class=""slug""><a href=""https://www.npr.org/sections/asia/"">Asia</a></h3>\n</div>\n<h2 class=""title""><a data-metrics='{""action"":""Click Story Title"",""category"":""Story List""}' href=""https://www.npr.org/2021/03/13/976805843/several-more-protesters-die-in-myanmar-quad-leaders-urge-restoration-of-democrac"">Several Mor..."
2,https://www.npr.org/sections/world/archive?start=16&date=3-31-2021,"<div class=""item-info-wrap"">\n<div class=""item-info"">\n<div class=""slug-wrap"">\n<h3 class=""slug""><a href=""https://www.npr.org/sections/middle-east/"">Middle East</a></h3>\n</div>\n<h2 class=""title""><a data-metrics='{""action"":""Click Story Title"",""category"":""Story List""}' href=""https://www.npr.org/2021/03/13/976785250/how-syrian-conflict-may-have-strengthened-the-influence-of-the-countrys-first-l..."
3,https://www.npr.org/sections/world/archive?start=16&date=3-31-2021,"<div class=""item-info-wrap"">\n<div class=""item-info"">\n<div class=""slug-wrap"">\n<h3 class=""slug""><a href=""https://www.npr.org/sections/africa/"">Africa</a></h3>\n</div>\n<h2 class=""title""><a data-metrics='{""action"":""Click Story Title"",""category"":""Story List""}' href=""https://www.npr.org/2021/03/13/976651242/the-stories-are-heartbreaking-what-1-reporter-witnessed-in-mozambiques-violence"">'The Sto..."
4,https://www.npr.org/sections/world/archive?start=16&date=3-31-2021,"<div class=""item-info-wrap"">\n<div class=""item-info"">\n<div class=""slug-wrap"">\n<h3 class=""slug""><a href=""https://www.npr.org/sections/asia/"">Asia</a></h3>\n</div>\n<h2 class=""title""><a data-metrics='{""action"":""Click Story Title"",""category"":""Story List""}' href=""https://www.npr.org/2021/03/12/976589506/myanmar-protest-leader-wants-the-military-out-and-isnt-interested-in-negotiating"">Myanmar Pro..."
...,...,...
136825,https://www.npr.org/sections/national/archive?start=144016&date=3-31-2021,"<div class=""item-info-wrap"">\n<div class=""item-info"">\n<div class=""slug-wrap"">\n<h3 class=""slug""><a href=""https://www.npr.org/sections/law/"">Law</a></h3>\n</div>\n<h2 class=""title""><a data-metrics='{""action"":""Click Story Title"",""category"":""Story List""}' href=""https://www.npr.org/templates/story/story.php?storyId=4717901"">Defending Military Personnel</a></h2>\n<p class=""teaser""><a href=""https:/..."
136826,https://www.npr.org/sections/national/archive?start=144016&date=3-31-2021,"<div class=""item-info-wrap"">\n<div class=""item-info"">\n<h2 class=""title""><a data-metrics='{""action"":""Click Story Title"",""category"":""Story List""}' href=""https://www.npr.org/templates/story/story.php?storyId=4717892"">Schwarzenegger's Popularity Dips</a></h2>\n<p class=""teaser""><a href=""https://www.npr.org/templates/story/story.php?storyId=4717892""><time datetime=""2005-06-24""><span class=""date"">J..."
136827,https://www.npr.org/sections/national/archive?start=144016&date=3-31-2021,"<div class=""item-info-wrap"">\n<div class=""item-info"">\n<div class=""slug-wrap"">\n<h3 class=""slug""><a href=""https://www.npr.org/sections/news/"">News</a></h3>\n</div>\n<h2 class=""title""><a data-metrics='{""action"":""Click Story Title"",""category"":""Story List""}' href=""https://www.npr.org/templates/story/story.php?storyId=4717883"">Bush and Jaafari: No Deadline on Leaving Iraq</a></h2>\n<p class=""tea

In [5]:
df = pd.read_csv('npr_politic_raw.csv', header=[0])
df = df[~df.rows.isnull()].reset_index(drop=True)
df

,url,rows
0,https://www.npr.org/sections/politics/archive?start=16&date=3-31-2021,"<div class=""item-info-wrap"">\n<div class=""item-info"">\n<h2 class=""title""><a data-metrics='{""action"":""Click Story Title"",""category"":""Story List""}' href=""https://www.npr.org/2021/03/17/978363447/texas-man-arrested-with-guns-ammo-outside-of-vice-presidential-residence"">Texas Man Arrested With Guns, Ammo Outside Of Vice Presidential Residence</a></h2>\n<p class=""teaser""><a href=""https://www.npr.or..."
1,https://www.npr.org/sections/politics/archive?start=16&date=3-31-2021,"<div class=""item-info-wrap"">\n<div class=""item-info"">\n<h2 class=""title""><a data-metrics='{""action"":""Click Story Title"",""category"":""Story List""}' href=""https://www.npr.org/2021/03/17/977842441/house-renews-violence-against-women-act-but-senate-hurdles-remain"">House Renews Violence Against Women Act, But Senate Hurdles Remain</a></h2>\n<p class=""teaser""><a href=""https://www.npr.org/2021/03/17/9..."
2,https://www.npr.org/sections/politics/archive?start=16&date=3-31-2021,"<div class=""item-info-wrap"">\n<div class=""item-info"">\n<div class=""slug-wrap"">\n<h3 class=""slug""><a href=""https://www.npr.org/sections/medical/"">Medical Treatments</a></h3>\n</div>\n<h2 class=""title""><a data-metrics='{""action"":""Click Story Title"",""category"":""Story List""}' href=""https://www.npr.org/2021/03/17/978288331/how-to-get-trump-voters-vaccinated-when-half-say-they-dont-plan-to"">How To G..."
3,https://www.npr.org/sections/politics/archive?start=16&date=3-31-2021,"<div class=""item-info-wrap"">\n<div class=""item-info"">\n<div class=""slug-wrap"">\n<h3 class=""slug""><a href=""https://www.npr.org/sections/coronavirus-live-updates/"">Coronavirus Updates</a></h3>\n</div>\n<h2 class=""title""><a data-metrics='{""action"":""Click Story Title"",""category"":""Story List""}' href=""https://www.npr.org/sections/coronavirus-live-updates/2021/03/17/978262865/white-house-announces-10..."
4,https://www.npr.org/sections/politics/archive?start=16&date=3-31-2021,"<div class=""item-info-wrap"">\n<div class=""item-info"">\n<h2 class=""title""><a data-metrics='{""action"":""Click Story Title"",""category"":""Story List""}' href=""https://www.npr.org/2021/03/17/978249859/irs-expected-to-push-back-tax-filing-deadline"">IRS Postpones Tax Filing Deadline To May 17</a></h2>\n<p class=""teaser""><a href=""https://www.npr.org/2021/03/17/978249859/irs-expected-to-push-back-tax-fili..."
...,...,...
44455,https://www.npr.org/sections/politics/archive?start=72208&date=3-31-2021,"<div class=""item-info-wrap"">\n<div class=""item-info"">\n<h2 class=""title""><a data-metrics='{""action"":""Click Story Title"",""category"":""Story List""}' href=""https://www.npr.org/templates/story/story.php?storyId=3413003"">Bush Faulted for NAACP Snub</a></h2>\n<p class=""teaser""><a href=""https://www.npr.org/templates/story/story.php?storyId=3413003""><time datetime=""2004-07-15""><span class=""date"">July 1..."
44456,https://www.npr.org/sections/politics/archive?start=72208&date=3-31-2021,"<div class=""item-info-wrap"">\n<div class=""item-info"">\n<h2 class=""title""><a data-metrics='{""action"":""Click Story Title"",""category"":""Story List""}' href=""https://www.npr.org/templates/story/story.php?storyId=3413001"">Bush Vies for More Support in Wisconsin</a></h2>\n<p class=""teaser""><a href=""https://www.npr.org/templates/story/story.php?storyId=3413001""><time datetime=""2004-07-15""><span class=""..."
44457,https://www.npr.org/sections/politics/archive?start=72208&date=3-31-2021,"<div class=""item-info-wrap"">\n<div class=""item-info"">\n<h2 class=""title""><a data-metrics='{""action"":""Click Story Title"",""category"":""Story List""}' href=""https://www.npr.org/templates/story/story.php?storyId=3399027"">Senate: No Gay Marriage Ban</a></h2>\n<p class=""teaser""><a href=""https://www.npr.org/templates/story/story.php?storyId=3399027""><time datetime=""2004-07-14""><span class=""date"">July 

In [11]:
df_merge = pd.merge(df_others, df, how='outer', indicator='Exist')
df_merge

,url,rows,Exist
0,https://www.npr.org/sections/world/archive?start=16&date=3-31-2021,"<div class=""item-info-wrap"">\n<div class=""item-info"">\n<div class=""slug-wrap"">\n<h3 class=""slug""><a href=""https://www.npr.org/sections/middle-east/"">Middle East</a></h3>\n</div>\n<h2 class=""title""><a data-metrics='{""action"":""Click Story Title"",""category"":""Story List""}' href=""https://www.npr.org/2021/03/13/976875790/marking-10-years-of-war-in-syria"">Marking 10 Years Of War In Syria</a></h2>\n<p...",left_only
1,https://www.npr.org/sections/world/archive?start=16&date=3-31-2021,"<div class=""item-info-wrap"">\n<div class=""item-info"">\n<div class=""slug-wrap"">\n<h3 class=""slug""><a href=""https://www.npr.org/sections/asia/"">Asia</a></h3>\n</div>\n<h2 class=""title""><a data-metrics='{""action"":""Click Story Title"",""category"":""Story List""}' href=""https://www.npr.org/2021/03/13/976805843/several-more-protesters-die-in-myanmar-quad-leaders-urge-restoration-of-democrac"">Several Mor...",left_only
2,https://www.npr.org/sections/world/archive?start=16&date=3-31-2021,"<div class=""item-info-wrap"">\n<div class=""item-info"">\n<div class=""slug-wrap"">\n<h3 class=""slug""><a href=""https://www.npr.org/sections/middle-east/"">Middle East</a></h3>\n</div>\n<h2 class=""title""><a data-metrics='{""action"":""Click Story Title"",""category"":""Story List""}' href=""https://www.npr.org/2021/03/13/976785250/how-syrian-conflict-may-have-strengthened-the-influence-of-the-countrys-first-l...",left_only
3,https://www.npr.org/sections/world/archive?start=16&date=3-31-2021,"<div class=""item-info-wrap"">\n<div class=""item-info"">\n<div class=""slug-wrap"">\n<h3 class=""slug""><a href=""https://www.npr.org/sections/africa/"">Africa</a></h3>\n</div>\n<h2 class=""title""><a data-metrics='{""action"":""Click Story Title"",""category"":""Story List""}' href=""https://www.npr.org/2021/03/13/976651242/the-stories-are-heartbreaking-what-1-reporter-witnessed-in-mozambiques-violence"">'The Sto...",left_only
4,https://www.npr.org/sections/world/archive?start=16&date=3-31-2021,"<div class=""item-info-wrap"">\n<div class=""item-info"">\n<div class=""slug-wrap"">\n<h3 class=""slug""><a href=""https://www.npr.org/sections/asia/"">Asia</a></h3>\n</div>\n<h2 class=""title""><a data-metrics='{""action"":""Click Story Title"",""category"":""Story List""}' href=""https://www.npr.org/2021/03/12/976589506/myanmar-protest-leader-wants-the-military-out-and-isnt-interested-in-negotiating"">Myanmar Pro...",left_only
...,...,...,...
181285,https://www.npr.org/sections/politics/archive?start=72208&date=3-31-2021,"<div class=""item-info-wrap"">\n<div class=""item-info"">\n<h2 class=""title""><a data-metrics='{""action"":""Click Story Title"",""category"":""Story List""}' href=""https://www.npr.org/templates/story/story.php?storyId=3413003"">Bush Faulted for NAACP Snub</a></h2>\n<p class=""teaser""><a href=""https://www.npr.org/templates/story/story.php?storyId=3413003""><time datetime=""2004-07-15""><span class=""date"">July 1...",right_only
181286,https://www.npr.org/sections/politics/archive?start=72208&date=3-31-2021,"<div class=""item-info-wrap"">\n<div class=""item-info"">\n<h2 class=""title""><a data-metrics='{""action"":""Click Story Title"",""category"":""Story List""}' href=""https://www.npr.org/templates/story/story.php?storyId=3413001"">Bush Vies for More Support in Wisconsin</a></h2>\n<p class=""teaser""><a href=""https://www.npr.org/templates/story/story.php?storyId=3413001""><time datetime=""2004-07-15""><span class=""...",right_only
181287,https://www.npr.org/sections/politics/archive?start=72208&date=3-31-2021,"<div class=""item-info-wrap"">\n<div class=""item-info"">\n<h2 class=""title""><a data-metrics='{""action"":""Click Story Title"",""category"":""Story List""}' href=""https://www.npr.org/templates/story/story.php?storyId=3399027"">Senate: No Gay Marriage Ban</a></h2>\n<p class=""teaser""><a href=""https://www.npr.org/templates/story/story

In [12]:
df_merge.Exist.unique()

['left_only', 'right_only']
Categories (2, object): ['left_only', 'right_only']

In [13]:
pandarallel.initialize(progress_bar=True)
df[['title', 'abstract', 'link']] = df['rows'].parallel_apply(extract_row)
df

INFO: Pandarallel will run on 24 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


,url,rows,title,abstract,link
0,https://www.npr.org/sections/politics/archive?start=16&date=3-31-2021,"<div class=""item-info-wrap"">\n<div class=""item-info"">\n<h2 class=""title""><a data-metrics='{""action"":""Click Story Title"",""category"":""Story List""}' href=""https://www.npr.org/2021/03/17/978363447/texas-man-arrested-with-guns-ammo-outside-of-vice-presidential-residence"">Texas Man Arrested With Guns, Ammo Outside Of Vice Presidential Residence</a></h2>\n<p class=""teaser""><a href=""https://www.npr.or...","Texas Man Arrested With Guns, Ammo Outside Of Vice Presidential Residence",The San Antonio man faces several weapons charges after police recovered a rifle and ammunition from his vehicle.,https://www.npr.org/2021/03/17/978363447/texas-man-arrested-with-guns-ammo-outside-of-vice-presidential-residence
1,https://www.npr.org/sections/politics/archive?start=16&date=3-31-2021,"<div class=""item-info-wrap"">\n<div class=""item-info"">\n<h2 class=""title""><a data-metrics='{""action"":""Click Story Title"",""category"":""Story List""}' href=""https://www.npr.org/2021/03/17/977842441/house-renews-violence-against-women-act-but-senate-hurdles-remain"">House Renews Violence Against Women Act, But Senate Hurdles Remain</a></h2>\n<p class=""teaser""><a href=""https://www.npr.org/2021/03/17/9...","House Renews Violence Against Women Act, But Senate Hurdles Remain","As a senator, President Biden played a key role in passing the landmark 1994 law, which he called ""one of my proudest legislative achievements."" It wasn't reauthorized in 2018 over partisan disputes.",https://www.npr.org/2021/03/17/977842441/house-renews-violence-against-women-act-but-senate-hurdles-remain
2,https://www.npr.org/sections/politics/archive?start=16&date=3-31-2021,"<div class=""item-info-wrap"">\n<div class=""item-info"">\n<div class=""slug-wrap"">\n<h3 class=""slug""><a href=""https://www.npr.org/sections/medical/"">Medical Treatments</a></h3>\n</div>\n<h2 class=""title""><a data-metrics='{""action"":""Click Story Title"",""category"":""Story List""}' href=""https://www.npr.org/2021/03/17/978288331/how-to-get-trump-voters-vaccinated-when-half-say-they-dont-plan-to"">How To G...",How To Get Trump Voters Vaccinated When Half Say They Don't Plan To,Nearly half of Trump voters say they don't plan to get vaccinated for COVID-19. The race is on to find pro-vaccine messaging that connects with vaccine-skeptical white Republicans.,https://www.npr.org/2021/03/17/978288331/how-to-get-trump-voters-vaccinated-when-half-say-they-dont-plan-to
3,https://www.npr.org/sections/politics/archive?start=16&date=3-31-2021,"<div class=""item-info-wrap"">\n<div class=""item-info"">\n<div class=""slug-wrap"">\n<h3 class=""slug""><a href=""https://www.npr.org/sections/coronavirus-live-updates/"">Coronavirus Updates</a></h3>\n</div>\n<h2 class=""title""><a data-metrics='{""action"":""Click Story Title"",""category"":""Story List""}' href=""https://www.npr.org/sections/coronavirus-live-updates/2021/03/17/978262865/white-house-announces-10...",White House Announces $10 Billion For COVID-19 Testing In Schools,"The new funds will enable K-12 schools to ramp up screening testing, which can ""identify asymptomatic disease and prevent clusters before they start,"" said CDC Director Rochelle Walensky.",https://www.npr.org/sections/coronavirus-live-updates/2021/03/17/978262865/white-house-announces-10-billion-for-covid-19-testing-in-schools
4,https://www.npr.org/sections/politics/archive?start=16&date=3-31-2021,"<div class=""item-info-wrap"">\n<div class=""item-info"">\n<h2 class=""title""><a data-metrics='{""action"":""Click Story Title"",""category"":""Story List""}' href=""https://www.npr.org/2021/03/17/978249859/irs-expected-to-push-back-tax-filing-deadline"">IRS Postpones Tax Filing Deadline To May 17</a></h2>\n<p class=""teaser""><a href=""https://www.npr.org/2021/03/17/978249859/irs-expected-to-push-back-tax-fili...",IRS Postpones Tax Filing Deadline To May 17,It is the second 

In [14]:
df = df[~df.duplicated(subset='link')].reset_index(drop=True)
df = df.iloc[:,2:]
df

,title,abstract,link
0,"Texas Man Arrested With Guns, Ammo Outside Of Vice Presidential Residence",The San Antonio man faces several weapons charges after police recovered a rifle and ammunition from his vehicle.,https://www.npr.org/2021/03/17/978363447/texas-man-arrested-with-guns-ammo-outside-of-vice-presidential-residence
1,"House Renews Violence Against Women Act, But Senate Hurdles Remain","As a senator, President Biden played a key role in passing the landmark 1994 law, which he called ""one of my proudest legislative achievements."" It wasn't reauthorized in 2018 over partisan disputes.",https://www.npr.org/2021/03/17/977842441/house-renews-violence-against-women-act-but-senate-hurdles-remain
2,How To Get Trump Voters Vaccinated When Half Say They Don't Plan To,Nearly half of Trump voters say they don't plan to get vaccinated for COVID-19. The race is on to find pro-vaccine messaging that connects with vaccine-skeptical white Republicans.,https://www.npr.org/2021/03/17/978288331/how-to-get-trump-voters-vaccinated-when-half-say-they-dont-plan-to
3,White House Announces $10 Billion For COVID-19 Testing In Schools,"The new funds will enable K-12 schools to ramp up screening testing, which can ""identify asymptomatic disease and prevent clusters before they start,"" said CDC Director Rochelle Walensky.",https://www.npr.org/sections/coronavirus-live-updates/2021/03/17/978262865/white-house-announces-10-billion-for-covid-19-testing-in-schools
4,IRS Postpones Tax Filing Deadline To May 17,It is the second consecutive year the agency has given taxpayers more time to file their returns because of the coronavirus pandemic.,https://www.npr.org/2021/03/17/978249859/irs-expected-to-push-back-tax-filing-deadline
...,...,...,...
44455,Bush Faulted for NAACP Snub,"President Bush has drawn criticism for declining to address the NAACP's annual convention in Philadelphia. Critics say Bush's refusal reflects a disregard for African-American concerns. Republicans charge the NAACP has become a veritable wing of the Democratic Party. Hear NPR's Renee Montagne, talk show host Armstrong Williams and Bruce Payne of Duke University.",https://www.npr.org/templates/story/story.php?storyId=3413003
44456,Bush Vies for More Support in Wisconsin,"President Bush continues his Midwest swing with a campaign stop in Wisconsin, a state he lost to Al Gore by a fraction of the vote in 2000. Bush touts signs of an improving economy, and cites his efforts in the war on terror as a main reason he should be re-elected. Hear NPR's Andrea Seabrook.",https://www.npr.org/templates/story/story.php?storyId=3413001
44457,Senate: No Gay Marriage Ban,"The Senate ends the prospect of a constitutional amendment banning gay marriage, at least for this year. An attempt to cut off debate and proceed to a vote on the amendment won the backing of less than half the Senate, far short of the two-thirds needed to approve the amendment itself. Six Republicans voted with Democrats, defeating the effort 50-48. NPR's David Welna reports.",https://www.npr.org/templates/story/story.php?storyId=3399027
44458,Report: British Prewar Intel on Iraq WMDs Flawed,"NPR's Alex Chadwick talks to Jonathan Freedland of The Guardian newspaper in London about the Butler report, a critique of British pre-war intelligence about Iraq. The Butler report, released Wednesday, is fueling criticism of British Prime Minister Tony Blair's alliance with the Bush administration in the Iraq war.",https://www.npr.org/templates/story/story.php?storyId=3388106


In [15]:
pandarallel.initialize(progress_bar=True)
df[['text']] = df['link'].parallel_apply(extract_text)

INFO: Pandarallel will run on 24 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [16]:
df

,title,abstract,link,text
0,"Texas Man Arrested With Guns, Ammo Outside Of Vice Presidential Residence",The San Antonio man faces several weapons charges after police recovered a rifle and ammunition from his vehicle.,https://www.npr.org/2021/03/17/978363447/texas-man-arrested-with-guns-ammo-outside-of-vice-presidential-residence,"Texas Man Arrested With Guns, Ammo Outside Of Vice Presidential Residence\n\nEnlarge this image toggle caption Mark Wilson/Getty Images Mark Wilson/Getty Images\n\nA man was arrested Wednesday afternoon near the Naval Observatory and charged with several counts related to weapons and ammunition, according the Washington, D.C., Metropolitan Police Department. The Naval Observatory is the vice p..."
1,"House Renews Violence Against Women Act, But Senate Hurdles Remain","As a senator, President Biden played a key role in passing the landmark 1994 law, which he called ""one of my proudest legislative achievements."" It wasn't reauthorized in 2018 over partisan disputes.",https://www.npr.org/2021/03/17/977842441/house-renews-violence-against-women-act-but-senate-hurdles-remain,"House Renews Violence Against Women Act, But Senate Hurdles Remain\n\nEnlarge this image toggle caption J. Scott Applewhite/AP J. Scott Applewhite/AP\n\nUpdated at 7:13 p.m. ET\n\nThe House approved with bipartisan support a reauthorization of the Violence Against Women Act, a popular 1994 law that protects and provides resources for victims of domestic abuse and sexual violence. The measure p..."
2,How To Get Trump Voters Vaccinated When Half Say They Don't Plan To,Nearly half of Trump voters say they don't plan to get vaccinated for COVID-19. The race is on to find pro-vaccine messaging that connects with vaccine-skeptical white Republicans.,https://www.npr.org/2021/03/17/978288331/how-to-get-trump-voters-vaccinated-when-half-say-they-dont-plan-to,"How To Get Trump Voters Vaccinated When Half Say They Don't Plan To\n\nNearly half of Trump voters say they don't plan to get vaccinated for COVID-19. The race is on to find pro-vaccine messaging that connects with vaccine-skeptical white Republicans.\n\nMARY LOUISE KELLY, HOST:\n\nThere's been a lot of focus on how communities of color feel about COVID-19 vaccines. New polling, including from..."
3,White House Announces $10 Billion For COVID-19 Testing In Schools,"The new funds will enable K-12 schools to ramp up screening testing, which can ""identify asymptomatic disease and prevent clusters before they start,"" said CDC Director Rochelle Walensky.",https://www.npr.org/sections/coronavirus-live-updates/2021/03/17/978262865/white-house-announces-10-billion-for-covid-19-testing-in-schools,"White House Announces $10 Billion For COVID-19 Testing In Schools\n\nEnlarge this image toggle caption Suzanne Kreiter/Boston Globe via Getty Images Suzanne Kreiter/Boston Globe via Getty Images\n\nThe Biden administration announced Wednesday that it will spend $10 billion to expand testing for schools, to aid in the president's goal to get schools open once again.\n\nThe funds will come from ..."
4,IRS Postpones Tax Filing Deadline To May 17,It is the second consecutive year the agency has given taxpayers more time to file their returns because of the coronavirus pandemic.,https://www.npr.org/2021/03/17/978249859/irs-expected-to-push-back-tax-filing-deadline,"IRS Postpones Tax Filing Deadline To May 17\n\nEnlarge this image toggle caption J. David Ake/AP J. David Ake/AP\n\nIf you've been putting off filing your tax return this year, here's some good news. The IRS and Treasury Department have delayed the filing deadline by another month.\n\nThe new deadline to file will be May 17. ""This continues to be a tough time for many people, and the IRS wants..."
...,...,...,...,...
44455,Bush Faulted for NAACP Snub,"President Bush has drawn criticism for declining to address the NAACP's annual convention in Philadelphia. Critics say Bush's refusal reflects a disregard for African-American concerns. Republic

In [26]:
df.to_csv('npr_politics_final.csv', index=False)

In [23]:
df_others = pd.read_csv('npr_final.csv', header=[0])
df_others

,title,abstract,link,text
0,Marking 10 Years Of War In Syria,"A decade since the start of Syria's civil war, hundreds of thousands have died, millions have been displaced, and the war still isn't over — although the Syrian regime has the clear upper hand.",https://www.npr.org/2021/03/13/976875790/marking-10-years-of-war-in-syria,"Marking 10 Years Of War In Syria\n\nA decade since the start of Syria's civil war, hundreds of thousands have died, millions have been displaced, and the war still isn't over — although the Syrian regime has the clear upper hand.\n\nMICHEL MARTIN, HOST:\n\nIt may be hard to remember now, but a decade ago, in March 2011, a wave of pro-democracy uprisings was sweeping the world, particularly the..."
1,Several More Protesters Die In Myanmar; 'Quad' Leaders Urge Restoration Of Democracy,Unrest continues as protests challenge a Feb. 1 military coup. It's estimated more than 70 protesters have died in the past six weeks. Regional leaders said it was urgent that democracy be restored.,https://www.npr.org/2021/03/13/976805843/several-more-protesters-die-in-myanmar-quad-leaders-urge-restoration-of-democrac,"Several More Protesters Die In Myanmar; 'Quad' Leaders Urge Restoration Of Democracy\n\nEnlarge this image toggle caption AP AP\n\nDeadly unrest continues to grip Myanmar as ongoing protests challenge the country's Feb. 1 military coup and President Biden and regional leaders are urging the restoration of democracy in that country.\n\nSeveral more protesters were killed early Saturday, after 1..."
2,How Syrian Conflict May Have Strengthened The Influence Of The Country's First Lady,"NPR's Scott Simons asks Economist correspondent Nicolas Pelham about Syria's first lady, Asma al-Assad, and how she has profited during the decade-long conflict in her country.",https://www.npr.org/2021/03/13/976785250/how-syrian-conflict-may-have-strengthened-the-influence-of-the-countrys-first-la,"How Syrian Conflict May Have Strengthened The Influence Of The Country's First Lady\n\nNPR's Scott Simons asks Economist correspondent Nicolas Pelham about Syria's first lady, Asma al-Assad, and how she has profited during the decade-long conflict in her country.\n\nSCOTT SIMON, HOST:\n\nAt least 400,000 people have died in the Syrian conflict that is now 10 years old. More than 5.6 million ha..."
3,'The Stories Are Heartbreaking.' What 1 Reporter Witnessed In Mozambique's Violence,"Journalists are not usually granted access to Cabo Delgado in northern Mozambique, where for three years a campaign of violence by Islamist extremists has left some 668,000 people displaced.",https://www.npr.org/2021/03/13/976651242/the-stories-are-heartbreaking-what-1-reporter-witnessed-in-mozambiques-violence,"'The Stories Are Heartbreaking.' What 1 Reporter Witnessed In Mozambique's Violence\n\nEnlarge this image toggle caption Marco Longari/AFP via Getty Images Marco Longari/AFP via Getty Images\n\nFor more than three years, northern Mozambique has been ravaged by violence and destruction, as a local Islamic insurgency has grown in intensity and brutality. More than 1,300 civilians have been kille..."
4,Myanmar Protest Leader Wants The Military Out And Isn't Interested In Negotiating,"Violence amid rallies have continued in Myanmar since the military coup which started on Feb. 1. One of the leaders, a young woman, is in no mood to negotiate and wants the military to give up power.",https://www.npr.org/2021/03/12/976589506/myanmar-protest-leader-wants-the-military-out-and-isnt-interested-in-negotiating,"Myanmar Protest Leader Wants The Military Out And Isn't Interested In Negotiating\n\nViolence amid rallies have continued in Myanmar since the military coup which started on Feb. 1. One of the leaders, a young woman, is in no mood to negotiate and wants the military to give up power.\n\nMARY LOUISE KELLY, HOST:\n\nToday the Biden administration said it will grant temporary protected status to ..."
...,...,...,...,...
131943,Defending Military Personnel,

In [30]:
# print('duplicated',df[df.link.isin(df_others.link)].shape )

# print('duplicated',df[df.link.isin(df_others.link)].shape )
# df  =  df[~df.rows.isin(df_others.rows)].reset_index(drop=True)
# df

In [28]:
df_merge = pd.merge(df_others, df, how='outer', indicator='Exist')
df_merge.Exist.value_counts()

left_only     113841
right_only     26353
both           18107
Name: Exist, dtype: int64

In [31]:
df_merge.to_csv('npr_merge_final.csv', index=False)